<a href="https://colab.research.google.com/github/karimghabbour/VirtualBuddy_RAG/blob/main/HPE_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Download GPT4ALL Model

In [1]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2024-03-25 08:49:57--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.0.159, 104.26.1.159, 172.67.71.169, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.0.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  72.0MB/s    in 62s     

2024-03-25 08:51:02 (57.9 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



#Step 2:Download Documents to be Accessed

In [4]:
#Mount Google Drive For Access
from google.colab import drive
drive.mount('/content/drive')

# Define the base path to the folder containing your PDFs
base_path = '/content/drive/My Drive/BasicRAG/'

# List of your PDF files
pdf_files = [
    'Resources inside HPE Barcelona Hub.pdf',
    'uncover-hidden-insights-create-business-advantage.pdf',
    'HPE GreenLake FAQ for technology innovators frequently asked questions-a00107144enw.pdf',
    'HPE Tech Care Service-a00108652enw.pdf'
]

# Full paths to the PDF files
pdf_paths = [base_path + pdf for pdf in pdf_files]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
